# Source localization with MNE/dSPM/sLORETA

The aim of this lecture is to teach you how to compute and apply
a linear inverse method such as MNE/dSPM/sLORETA on evoked/raw/epochs data.

`
Authors: Alexandre Gramfort <alexandre.gramfort@inria.fr>
         Denis Engemann <denis.engemann@gmail.com>
         Richard Hoechenberger <richard.hoechenberger@gmail.com>
`

In [ ]:
# add plot inline in the page
%matplotlib inline

import numpy as np
import matplotlib.pyplot as plt

import mne
mne.set_log_level('WARNING')

## Get epochs (from first notebook)

In [ ]:
from pathlib import Path
import mne
from mne_bids import BIDSPath

bids_root = Path('ds000248/')
deriv_root = bids_root / 'derivatives' 

bp = BIDSPath(
    root=deriv_root,  # BIDS derivatives folder
    subject='01',  # subject name as a string
    datatype='meg',  # datatype (meg, eeg, ieeg, anat, etc.)
    task='audiovisual',  # Task e.g. audiovisual, rest, etc.
    run='01',  # run id (optional)
    session=None,  # there is no session on this dataset
    suffix="epo",
    extension='.fif',
    check=False
)

epochs_fname = bp.fpath
epochs_fname

In [ ]:
epochs = mne.read_epochs(epochs_fname)

## Compute noise covariance

See:

Engemann D.A., Gramfort A., Automated model selection in covariance estimation and spatial whitening of MEG and EEG signals, Neuroimage. 2015 Mar

In [ ]:
noise_cov = mne.compute_covariance(epochs, tmax=0.,
                                   method=['shrunk', 'empirical'])
print(noise_cov.data.shape)

In [ ]:
mne.viz.plot_cov(noise_cov, epochs.info)

## Compute the evoked response

In [ ]:
evoked = epochs.average()
evoked.plot()
evoked.plot_topomap(times=np.linspace(0.05, 0.15, 5), ch_type='mag');

# Show whitening

In [ ]:
%matplotlib inline
evoked.plot_white(noise_cov);

## Inverse modeling with MNE and dSPM on evoked and raw data

Import the required functions:

In [ ]:
from mne.forward import read_forward_solution
from mne.minimum_norm import (make_inverse_operator, apply_inverse,
                              write_inverse_operator)

## Read the forward solution and compute the inverse operator

MNE/dSPM/sLORETA lead to linear inverse models that are independant
from the data (as they use just the noise covariance) and can therefore be
precomputed and applied to the data at a later stage.

In [ ]:
bp_fwd = bp.copy().update(
    root=deriv_root,
    suffix='fwd',
    task=None,
    run=None,
    check=False
)

fwd = mne.read_forward_solution(bp_fwd)
fwd = mne.convert_forward_solution(fwd, surf_ori=True)

# Restrict forward solution as necessary for MEG
fwd = mne.pick_types_forward(fwd, meg=True, eeg=False)

# make an M/EEG, MEG-only, and EEG-only inverse operator
info = evoked.info
inverse_operator = make_inverse_operator(info, fwd, noise_cov,
                                         loose=0.2, depth=0.8)

write_inverse_operator('sample_audvis-meg-oct-6-inv.fif',
                       inverse_operator)

## Compute inverse solution / Apply inverse operators

In [ ]:
method = "dSPM"
snr = 3.
lambda2 = 1. / snr ** 2
stc = apply_inverse(evoked, inverse_operator, lambda2,
                    method=method, pick_ori=None)
print(stc)

In [ ]:
stc.data.shape

In [ ]:
stc.save('fixed_ori')

In [ ]:
# make one with no orientation constraint (free orientation)
# inverse_operator = make_inverse_operator(info, fwd, noise_cov,
#                                          loose=1., depth=0.8)
# stc = apply_inverse(evoked, inverse_operator, lambda2,
#                     method=method, pick_ori=None)
# stc.save('free_ori')

The ``stc`` (Source Time Courses) are defined on a source space formed by 7498 candidate
locations and for a duration spanning 106 time points.

## Show the result

In [ ]:
subjects_dir = bids_root / 'derivatives' / 'freesurfer' / 'subjects'

brain = stc.plot(surface='inflated', hemi='rh', subjects_dir=subjects_dir)
brain.save_image('dspm.jpg')

## Morphing data to an average brain for group studies

In [ ]:
subject = "sub-01"

morph = mne.compute_source_morph(stc, subject_from=subject,
                                 subject_to='fsaverage',
                                 subjects_dir=subjects_dir)
stc_fsaverage = morph.apply(stc)

In [ ]:
stc_fsaverage.save('fsaverage_dspm')

In [ ]:
brain_fsaverage = stc_fsaverage.plot(surface='inflated', hemi='rh',
                                     subjects_dir=subjects_dir)
brain_fsaverage.save_image('dspm_fsaverage.jpg')

## Exercises
- Run sLORETA on the same data and compare source localizations
- Run an LCMV beamformer on the same data and compare source localizations

### Going further:

- https://mne.tools/dev/auto_tutorials/index.html#source-localization-and-inverses
- https://mne.tools/dev/auto_examples/inverse/compute_mne_inverse_epochs_in_label.html
- https://mne.tools/dev/auto_examples/inverse/plot_label_source_activations.html
- https://mne.tools/dev/auto_examples/inverse/plot_label_from_stc.html